In [36]:
import argparse
from argparse import Namespace

args = Namespace()
args.data_dir = '/projects/grail/portrait-rephotography/dataset/aligned2/historical_samples'
args.out_dir_fmt = '../results/resolution-test/historical_samples{}'
args.sizes = [64, 128, 256, 512]
args.median_filter = 0

In [37]:
import os
from os.path import join as pjoin
from shutil import copyfile

import cv2


opt_str = ""
if args.median_filter > 0:
    opt_str = f"-med{args.median_filter}"
args.out_dir_fmt += opt_str

for size in args.sizes + ['']:
    os.makedirs(args.out_dir_fmt.format(size), exist_ok=True)

    
names = os.listdir(args.data_dir)
for name in names:
    in_fn = pjoin(args.data_dir, name)
    im = cv2.imread(in_fn)
    large_dir = args.out_dir_fmt.format('')
    copyfile(in_fn, pjoin(large_dir, name))
    
    if args.median_filter > 0:
        im = cv2.medianBlur(im, args.median_filter)
    
    H, W = im.shape[:2]
    pre, ext = os.path.splitext(name)
    for size in args.sizes:
        small = cv2.resize(im, (size, size), interpolation=cv2.INTER_AREA)
        small_fn = pjoin(args.out_dir_fmt.format(size), name)
        cv2.imwrite(small_fn, small)
        
        upsample = cv2.resize(small, (W, H), interpolation=cv2.INTER_CUBIC)
        up_fn = pjoin(large_dir, f'{pre}-{size}{opt_str}{ext}')
        cv2.imwrite(up_fn, upsample)
        
        diff = (upsample - im) + 127
        diff_fn = pjoin(large_dir, f'{pre}-{size}{opt_str}-diff{ext}')
        cv2.imwrite(diff_fn, diff)